In [1]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [2]:
# from langchain.llms import OpenAI # this code has been deprecated since recording.
from langchain.chat_models import ChatOpenAI # this is the replacement
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from random import sample

from langchain.document_loaders.csv_loader import CSVLoader

First, let's create a loader and load reviews from tv-reviews.csv into memory

In [3]:
# TODO: load reviews from tv-reviews.csv
loader = CSVLoader(file_path='./tv-reviews.csv')
data = loader.load()
# print(data)

Then, let's initialize our LLM

In [7]:
# TODO: initialize OpenAI object with your API key
model_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(model_name=model_name, temperature=0)

# class ReviewSentiment(BaseModel):
#     positives: List[NonNegativeInt] = Field(description="index of a positive TV review, starting from 0")
#     negatives: List[NonNegativeInt] = Field(description="index of a positive TV review, starting from 0")

# parser = PydanticOutputParser(pydantic_object=ReviewSentiment)
# print(parser.get_format_instructions())

Now, let's setup our parser and a template  - 

**Note**  that since recording, the code to initialize the model has been updated to 

`llm = ChatOpenAI()`

In [5]:
class ReviewSentiment(BaseModel):
    positives: List[NonNegativeInt] = Field(description="index of a positive TV review, starting from 0")
    negatives: List[NonNegativeInt] = Field(description="index of a negative TV review, starting from 0")
        
parser = PydanticOutputParser(pydantic_object=ReviewSentiment)
# print(parser.get_format_instructions())
# TODO: setup a template with partial and input variables

Pick 3 sample reviews to classify - LLMs have a limited context window they can work with. In later exercises, we'll see how to deal with that differently

In [8]:
# TODO: pick 3 random reviews and save them into reviews_to_classify variable
prompt = PromptTemplate(
    template="{question}\n{format_instructions}\nContext: {context}",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

In [11]:
question = """
    Classify TV reviews provided in the context into positive and negative. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""

In [12]:
reviews_to_classify = sample(data, 3)
context = '\n'.join(review.page_content for review in reviews_to_classify)

query = prompt.format(context=context, question=question)
print(query)


    Classify TV reviews provided in the context into positive and negative. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"title": "Positives", "description": "index of a positive TV review, starting from 0", "type": "array", "items": {"type": "integer", "minimum": 0}}, "negatives": {"title": "Negatives", "description": "index of a positive TV 

# generate textual prompt from the prompt template
question = """
    Review TVs provided in the context. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""
query = prompt.format(context = context, question = question)

Finally, let's send our query to LLM and use the parser we setup to parse an output into a Python object

**NOTE**: Since recording the code to feed the query to the llm has been updated to

`llm.predict(query)`

In [14]:
# TODO: query LLM, then parse output into the result variable
output = llm.predict(query)
print(output)

result = parser.parse(output)
print(result)

print("Positives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.positives]))
print("Negatives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.negatives]))

{
    "positives": [0, 1, 2],
    "negatives": []
}
positives=[0, 1, 2] negatives=[]
Positives:
TV Name: VisionMax Ultra
Review Title: Unmatched Visuals
Review Rating: 10
Review Text: The VisionMax Ultra lives up to its name by delivering unmatched visuals. The 4K resolution and HDR technology bring every scene to life with incredible detail and vibrant colors. It's like having a cinema in my own home. I couldn't be happier with this TV!
TV Name: Imagix Pro
Review Title: Stunning Design
Review Rating: 10
Review Text: The Imagix Pro not only offers exceptional performance but also boasts a stunning design. The ultra-thin bezels and sleek finish make it a focal point of my living room. The TV blends seamlessly with my decor. It's a true work of art!
TV Name: Imagix Pro
Review Title: Perfect Gaming TV
Review Rating: 8
Review Text: As a gaming enthusiast, the Imagix Pro is a dream come true. The low input lag and high refresh rate ensure smooth gameplay. The colors are vibrant and the deta